<a href="https://colab.research.google.com/github/drew0523/MIR-Music_Information_Retrieval/blob/main/notebooks/4th_week_genre_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genre Classification with GTZAN dataset

In [ ]:
!pip install --upgrade gdown
!gdown 1-4elQY1C-n23u3QqomnLiI9CN9iPrWC3
!tar -xzf genres.tar.gz


Downloading...
From (original): https://drive.google.com/uc?id=1-4elQY1C-n23u3QqomnLiI9CN9iPrWC3
From (redirected): https://drive.google.com/uc?id=1-4elQY1C-n23u3QqomnLiI9CN9iPrWC3&confirm=t&uuid=146bb1b5-4863-486e-9d49-9125620ae04e
To: /content/genres.tar.gz
100% 1.23G/1.23G [00:15<00:00, 78.8MB/s]
